# Script for writing all possible binary permutations of a certain # of bits to a csv file

In [3]:
import numpy as np
import pandas as pd

decList = np.arange(0,256,1) # make sure to go to 2^N+1 so capture the last entry (all 1s)
for i in decList:
    binStr = format(i,"08b")
    if i == 0:
        df = pd.DataFrame({"Code": [binStr]})
    else:
        df2 = pd.DataFrame({"Code": [binStr]})
        df = pd.concat([df, df2],ignore_index=True)
df.to_csv("allPossible8bitWords.csv",index=False)       